### Elasticsearch 查询操作
本笔记本演示如何使用 Python 进行 Elasticsearch 的各种查询操作


In [63]:
from elasticsearch_dsl import Search,Q,connections,Index

connection = connections.create_connection(hosts="http://localhost:9200");

### 基础查询
演示基本的 Match 查询，包括查询构建、执行和结果解析。


In [64]:
#创建一个查询对象
search = Search(index="books")

In [65]:
#添加一个match查询
query = Q('match',title='西游')
s = search.query(query)

In [66]:
response = s.execute()

In [67]:
response

<Response: [<Hit(books/6P52lZYBXvZuT4g9B-TA): {'title': '西游记', 'description': '由明朝作家吴承恩创作，讲述了孙悟空、猪八戒、沙僧等人与...}>]>

### 查询结果处理
展示如何获取查询结果的元数据（ID、评分）和文档内容。


In [68]:
# 打印原始查询DSL
print(response.to_dict()["hits"]["hits"])  # 查看所有命中文档

[{'_index': 'books', '_id': '6P52lZYBXvZuT4g9B-TA', '_score': 4.3191433, '_source': {'title': '西游记', 'description': '由明朝作家吴承恩创作，讲述了孙悟空、猪八戒、沙僧等人与唐僧师徒四人西天取经的故事。小说通过描绘师徒四人在取经路上所遭遇的种种艰难险阻，反映了人性的善恶、道德观念和宽容精神，成为中国文学的经典之作。', 'category': '小说', 'price': 10, 'in_stock': False}}]


In [69]:
# for hit in response.hits:#//print(1)
#     print(f'title:{hit.title}')
#     print(f'description:{hit.description}')
for hit in response.hits:
    print(f"ID: {hit.meta.id}, Score: {hit.meta.score}")  # 检查是否同一文档
    print(hit.to_dict())  # 打印完整文档

ID: 6P52lZYBXvZuT4g9B-TA, Score: 4.3191433
{'title': '西游记', 'description': '由明朝作家吴承恩创作，讲述了孙悟空、猪八戒、沙僧等人与唐僧师徒四人西天取经的故事。小说通过描绘师徒四人在取经路上所遭遇的种种艰难险阻，反映了人性的善恶、道德观念和宽容精神，成为中国文学的经典之作。', 'category': '小说', 'price': 10, 'in_stock': False}


### 重复数据处理
演示如何检测和处理索引中的重复文档。


In [33]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search

# 连接ES
es = Elasticsearch("http://localhost:9200")

# 找出所有《西游记》文档
duplicates = es.search(
    index="books",
    body={
        "query": {"match": {"title": "西游记"}},
        "_source": False,  # 不返回文档内容
        "size": 100
    }
)

# 保留第一条，删除其余（根据文档时间戳选择最新的一条）
keep_id = duplicates['hits']['hits'][0]['_id']  # 保留第一个文档ID
for doc in duplicates['hits']['hits'][1:]:
    es.delete(index="books", id=doc['_id'])

print(f"已清理重复数据，保留文档ID: {keep_id}")

已清理重复数据，保留文档ID: cQZhlJYBAwmkFhAi3VA3


In [70]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search

es = Elasticsearch(hosts="http://localhost:9200") 
s = Search(using=es, index="books").query("match", title="西游记")
response = s.execute()

print(f"找到 {response.hits.total.value} 条重复记录")
for hit in response:
    print(f"ID: {hit.meta.id}, 内容哈希: {hash(str(hit.to_dict()))}")

找到 1 条重复记录
ID: 6P52lZYBXvZuT4g9B-TA, 内容哈希: 8814111650751092178


In [98]:
from elasticsearch_dsl import Search, Q, connections

# 1. 建立连接
connections.create_connection(hosts="http://localhost:9200")

# 2. 创建查询对象
search = Search(index="books")  # 修正拼写错误：原图写成了 num="books"

# 3. 构建查询条件
query = Q('match', title="西游")  # 修正拼写错误：原图写成了 "西路"
search = search.query(query)

# 4. 执行查询
response = search.execute()

# 5. 打印结果
print("=== 命中文档 ===")
for hit in response:
    print(f"标题: {hit.title}")
    print(f"描述: {hit.description}\n")

# 6. 打印原始DSL查询语句
print("=== 原始查询DSL ===")
print(response.to_dict())

=== 命中文档 ===
标题: 西游记
描述: 由明朝作家吴承恩创作，讲述了孙悟空、猪八戒、沙僧等人与唐僧师徒四人西天取经的故事。小说通过描绘师徒四人在取经路上所遭遇的种种艰难险阻，反映了人性的善恶、道德观念和宽容精神，成为中国文学的经典之作。

=== 原始查询DSL ===
{'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 4.3191433, 'hits': [{'_index': 'books', '_id': '6P52lZYBXvZuT4g9B-TA', '_score': 4.3191433, '_source': {'title': '西游记', 'description': '由明朝作家吴承恩创作，讲述了孙悟空、猪八戒、沙僧等人与唐僧师徒四人西天取经的故事。小说通过描绘师徒四人在取经路上所遭遇的种种艰难险阻，反映了人性的善恶、道德观念和宽容精神，成为中国文学的经典之作。', 'category': '小说', 'price': 10, 'in_stock': False}}]}}


### 多重查询
使用 MultiSearch 同时执行多个查询并处理结果。
多重查询

In [36]:
from elasticsearch_dsl import MultiSearch

In [112]:
ms =MultiSearch(index='books')

#创建第一个查询
s1 = Search()
s1 = s1.query('match',title='西游')
ms = ms.add(s1)

#创建第二个查询
s2 = Search()
s2 = s2.query('match',title='水浒')
ms = ms.add(s2)

responses = ms.execute()

for response in responses:
    for hit in response.hits:
        print(hit.title)


西游记
水浒传


### 查询调试
展示如何查看原始查询 DSL 和调试查询语句。


In [105]:
from elasticsearch_dsl import MultiSearch, Search

ms = MultiSearch(index='books')

# 修正后的查询1：西游记
s1 = Search().query('match', title='西游')  # 使用完整书名
ms = ms.add(s1)

# 查询2：水浒传
s2 = Search().query('match', title='水浒')
ms = ms.add(s2)

responses = ms.execute()

for i, response in enumerate(responses):
    print(f"查询{i+1}结果:")
    for hit in response:
        print(hit.title)

查询1结果:
西游记
查询2结果:
水浒传
